In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# json파일들 -> csv파일 통합

In [ ]:
import json
import zipfile
import pandas as pd
myzip_r = zipfile.ZipFile('/content/drive/MyDrive/데이터/파/train/[라벨]파_0.정상.zip', 'r')
name_list = myzip_r.namelist()
df = pd.DataFrame()
for name in name_list:
    with myzip_r.open(name) as file:
        data = json.load(file)
        df2 = pd.DataFrame(data.values())
        df = pd.concat([df,df2])
# 이제 data_list에는 모든 파일의 데이터가 포함되어 있습니다.
df.loc[:,['disease', 'crop', 'area', 'grow', 'risk', 'points']] = df[['disease', 'crop', 'area', 'grow', 'risk', 'points']].shift(-1)
odd = []
for i in range(0,int(df.shape[0]/2)):
    odd.append(2*i)
data1 = df.iloc[odd,:]
data1.loc[:,'status'] = 0

<ipython-input-2-c2e2b0ccd2e0>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1.loc[:,'status'] = 0


In [ ]:
import json
import zipfile
import pandas as pd
myzip_r = zipfile.ZipFile('/content/drive/MyDrive/데이터/파/train/[라벨]파_1.질병.zip', 'r')
name_list = myzip_r.namelist()
df = pd.DataFrame()
for name in name_list:
    with myzip_r.open(name) as file:
        data = json.load(file)
        df2 = pd.DataFrame(data.values())
        df = pd.concat([df,df2])
# 이제 data_list에는 모든 파일의 데이터가 포함되어 있습니다.
df.loc[:,['disease', 'crop', 'area', 'grow', 'risk', 'points']] = df[['disease', 'crop', 'area', 'grow', 'risk', 'points']].shift(-1)
odd = []
for i in range(0,int(df.shape[0]/2)):
    odd.append(2*i)
data2 = df.iloc[odd,:]
data2.loc[:,'status'] = 1

<ipython-input-3-67c9637c439e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2.loc[:,'status'] = 1


In [ ]:
import json
import zipfile
import pandas as pd
myzip_r = zipfile.ZipFile('/content/drive/MyDrive/데이터/파/train/[라벨]파_9.증강.zip', 'r')
name_list = myzip_r.namelist()
df = pd.DataFrame()
for name in name_list:
    with myzip_r.open(name) as file:
        data = json.load(file)
        df2 = pd.DataFrame(data.values())
        df = pd.concat([df,df2])
df.loc[:,['disease', 'crop', 'area', 'grow', 'risk', 'points']] = df[['disease', 'crop', 'area', 'grow', 'risk', 'points']].shift(-1)
df.loc[:,['brightness', 'contrast', 'saturation', 'hue', 'transpose', 'hflip', 'vflip']] = df[['brightness', 'contrast', 'saturation', 'hue', 'transpose', 'hflip','vflip']].shift(-2)
odd = []
for i in range(0,int(df.shape[0]/3)):
    odd.append(3*i)
print(len(odd))
data3 = df.iloc[odd,:]
data3.loc[:,'status'] = 9

31755


<ipython-input-4-29d52ed443e8>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3.loc[:,'status'] = 9


In [ ]:
print(data1.shape,data2.shape,data3.shape)
data = pd.concat([data1,data2,data3])
data.info()

In [ ]:
data.drop('region',axis=1,inplace=True)

In [ ]:
data.to_csv('[라벨]파.csv',index=False)

# 넘파이 파일 불러와서 그레이스케일 변환 후 데이터분할

In [ ]:
!pip install Pillow

In [ ]:
import os,sys
from PIL import Image
def resize_and_crop(img_array,size):
    img = Image.fromarray(image_array)
    img_ratio = img.size[0] / float(img.size[1])
    ratio = size[0] / float(size[1])

    if ratio > img_ratio:
        img = img.resize((size[0], int(round(size[0] * img.size[1] / img.size[0]))),
            Image.LANCZOS)
        box = (0, int(round((img.size[1] - size[1]) / 2)), img.size[0],
               int(round((img.size[1] + size[1]) / 2)))
        img = img.crop(box)

    elif ratio < img_ratio:
        img = img.resize((int(round(size[1] * img.size[0] / img.size[1])), size[1]),
            Image.LANCZOS)
        box = (int(round((img.size[0] - size[0]) / 2)), 0,
                int(round((img.size[0] + size[0]) / 2)), img.size[1])
        img = img.crop(box)

    else :
        img = img.resize((size[0], size[1]), Image.LANCZOS)

    return img

In [ ]:
import pandas as pd
import zipfile
import numpy as np
from PIL import Image

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/데이터/파/train/[원천]파_0.정상.zip'
i = 1
# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내의 파일 목록을 가져옴
    name_list = z.namelist()
    image_arrays = []
    for name in name_list:
        with z.open(name) as file:
            image = Image.open(file)
            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            output_size = (224, 224)
            # 이미지 축소
            resized_img = resize_and_crop(image_array,output_size)
            # 그레이스케일
            img = np.array(resized_img).reshape(224, 224, 3)
            image_arrays.append(img)
        print(f'{len(name_list)}중에 {i}번 성공')
        i += 1
# 리스트의 numpy 배열들을 4차원 numpy 배열로 변환
image_arr = np.stack(image_arrays)
file_path = '/content/drive/MyDrive/데이터/파/train/파_정상_224_color.npy'
np.save(file_path, image_arr)
# 결과 확인
print(image_arr.shape)


In [ ]:
import pandas as pd
import zipfile
import numpy as np
from PIL import Image

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/데이터/파/train/[원천]파_1.질병.zip'
i = 1
# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내의 파일 목록을 가져옴
    name_list = z.namelist()
    image_arrays = []
    for name in name_list:
        with z.open(name) as file:
            image = Image.open(file)
            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            output_size = (224, 224)
            # 이미지 축소
            resized_img = resize_and_crop(image_array,output_size)
            # 그레이스케일
            img = np.array(resized_img).reshape(224,224,3)
            image_arrays.append(img)
        print(f'{len(name_list)}중에 {i}번 성공')
        i += 1
# 리스트의 numpy 배열들을 4차원 numpy 배열로 변환
image_arr = np.stack(image_arrays)
file_path = '/content/drive/MyDrive/데이터/파/train/파_질병_224_color.npy'
np.save(file_path, image_arr)
# 결과 확인
print(image_arr.shape)


In [ ]:
import pandas as pd
import zipfile
import numpy as np
from PIL import Image

# ZIP 파일 경로
zip_file_path = '/content/drive/MyDrive/데이터/파/train/[원천]파_9.증강.zip'
i = 1
# ZIP 파일 열기
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # ZIP 내의 파일 목록을 가져옴
    name_list = z.namelist()
    image_arrays = []
    for name in name_list:
        with z.open(name) as file:
            image = Image.open(file)
            # 이미지를 numpy 배열로 변환
            image_array = np.array(image)
            output_size = (224,224)
            # 이미지 축소
            resized_img = resize_and_crop(image_array,output_size)
            # 그레이스케일
            img = np.array(resized_img).reshape(224,224,3)
            image_arrays.append(img)
        print(f'{len(name_list)}중에 {i}번 성공')
        i += 1
# 리스트의 numpy 배열들을 4차원 numpy 배열로 변환
image_arr = np.stack(image_arrays)
file_path = '/content/drive/MyDrive/데이터/파/train/파_증강_224_color.npy'
np.save(file_path, image_arr)
# 결과 확인
print(image_arr.shape)


In [ ]:
import numpy as np
import pandas as pd
normal = np.load('/content/drive/MyDrive/데이터/파/train/파_정상_224.npy')
disease = np.load('/content/drive/My Drive/데이터/파/train/파_질병_224.npy')
expend = np.load('/content/drive/My Drive/데이터/파/train/파_증강_224.npy')
diseases = np.concatenate((disease, expend), axis=0)
X_train2 = np.concatenate((normal, diseases), axis=0)
labels = pd.read_csv('/content/drive/MyDrive/데이터/파/train/[라벨]파.csv')
if X_train2.shape[0]==labels.shape[0] :
    y_train = labels.loc[:,'disease']
else :
    print('길이가 같지 않습니다.')

In [ ]:
np.unique(y_train,return_counts=True)

In [ ]:
y_train[y_train==16]=1
y_train[y_train==17]=2
y_train[y_train==18]=3
np.unique(y_train,return_counts=True)

In [ ]:
num = 4144
a = np.where(y_train == 0)[0][:num]
b = np.where(y_train == 1)[0][:num]
c = np.where(y_train == 2)[0][:num]
d = np.where(y_train == 3)[0][:num]
select = []
select.extend(a)
select.extend(b)
select.extend(c)
select.extend(d)
X_train = X_train2[select]
y_train = y_train[select]


In [ ]:
if y_train.shape[0] == num*4 :
    print(y_train.shape)
else :
    print('길이가 다릅니다.')

In [ ]:
X_train.shape

In [ ]:
np.save('/content/drive/MyDrive/데이터/파/X_train_224_equal.npy',X_train)
np.save('/content/drive/MyDrive/데이터/파/y_train_224_equal.npy',y_train)